In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

In [ ]:
## Loading images and labels
(train_ds, train_labels), (test_ds, test_labels) = tfds.load(
    "tf_flowers",
    split=["train[:70%]", "train[:30%]"], ## Train test split
    batch_size=-1,
    as_supervised=True,  # Include labels
)


In [ ]:

## check existing image size
train_ds[0].shape


TensorShape([442, 1024, 3])

In [ ]:

## Resizing images
train_ds = tf.image.resize(train_ds, (150, 150))
test_ds = tf.image.resize(test_ds, (150, 150))

In [ ]:

train_labels

<tf.Tensor: shape=(2569,), dtype=int64, numpy=array([2, 3, 3, ..., 0, 2, 0])>

In [ ]:
## Transforming labels to correct format
train_labels = to_categorical(train_labels, num_classes=5)
test_labels = to_categorical(test_labels, num_classes=5)

In [ ]:
train_labels[0]

array([0., 0., 1., 0., 0.], dtype=float32)

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

In [ ]:
train_ds[0].shape

TensorShape([150, 150, 3])

In [ ]:
## Loading VGG16 model
base_model = VGG16(weights="imagenet", include_top=False, input_shape=train_ds[0].shape)

In [ ]:
## will not train base mode
base_model.trainable = False

In [ ]:
## Preprocessing input
train_ds = preprocess_input(train_ds)
test_ds = preprocess_input(test_ds)

In [ ]:
## model details
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [ ]:
#add our layers on top of this model
from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(50, activation='relu')
dense_layer_2 = layers.Dense(20, activation='relu')
prediction_layer = layers.Dense(5, activation='softmax')


model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

In [ ]:
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

In [ ]:
model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=32, callbacks=[es])

Epoch 1/50
65/65 [==============================] - 600s 9s/step - loss: 2.1488 - accuracy: 0.4672 - val_loss: 1.1037 - val_accuracy: 0.5973
Epoch 2/50
65/65 [==============================] - 591s 9s/step - loss: 0.7791 - accuracy: 0.7109 - val_loss: 0.9647 - val_accuracy: 0.6342
Epoch 3/50
65/65 [==============================] - 591s 9s/step - loss: 0.4988 - accuracy: 0.8229 - val_loss: 1.1163 - val_accuracy: 0.6459
Epoch 4/50
65/65 [==============================] - 588s 9s/step - loss: 0.3618 - accuracy: 0.8696 - val_loss: 1.1657 - val_accuracy: 0.6790
Epoch 5/50
65/65 [==============================] - 590s 9s/step - loss: 0.2401 - accuracy: 0.9173 - val_loss: 1.2360 - val_accuracy: 0.6693
Epoch 6/50
65/65 [==============================] - 587s 9s/step - loss: 0.1545 - accuracy: 0.9484 - val_loss: 1.2960 - val_accuracy: 0.6790
Epoch 7/50
65/65 [==============================] - 588s 9s/step - loss: 0.0910 - accuracy: 0.9776 - val_loss: 1.3971 - val_accuracy: 0.6673
Epoch 8/50
65

In [ ]:
#Model Build from Scratch
from tensorflow.keras import Sequential, layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Rescaling


full_model = Sequential()
full_model.add(Rescaling(1./255, input_shape=(150,150,3)))

full_model.add(layers.Conv2D(16, kernel_size=10, activation='relu'))
full_model.add(layers.MaxPooling2D(3))

full_model.add(layers.Conv2D(32, kernel_size=8, activation="relu"))
full_model.add(layers.MaxPooling2D(2))

full_model.add(layers.Conv2D(32, kernel_size=6, activation="relu"))
full_model.add(layers.MaxPooling2D(2))

full_model.add(layers.Flatten())
full_model.add(layers.Dense(50, activation='relu'))
full_model.add(layers.Dense(20, activation='relu'))
full_model.add(layers.Dense(5, activation='softmax'))


full_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)


es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

full_model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=32, callbacks=[es])